# Predict the rank of elliptic curves

In this notebook, we will build a logistic regression model that predicts the rank of the elliptic curve, where Frobenius traces $a_p(E)$ are used as features.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import polars as pl
from tqdm import tqdm

from lmf import db

In [2]:
# Elliptic curve database
ec_db = db.ec_curvedata

In [3]:
def query_data(N1, N2, r=None):
    # Elliptic curves over Q of conductor N1 <= N <= N2
    if r is None:
        return list(ec_db.search({"conductor": {"$gte": N1, "$lte": N2}}))
    return list(ec_db.search({"conductor": {"$gte": N1, "$lte": N2}, "rank": r}))

def get_df(max_N, num_ap, r=None, save_csv=False):
    """
    Get a polars dataframe of elliptic curves over Q of conductor up to max_N,
    including the first num_ap coefficients a_p of the L-series.
    If r is specified, then we assume that it is a list of ranks to filter.
    If save_csv is True, save the dataframe to a CSV file.
    """
    data = query_data(1, max_N)
    pmax = Primes()[num_ap - 1]
    isog_labels = set()

    columns = ["isog_label", "rank"]
    for p in Primes()[:num_ap]:
        columns.append(f"a_{p:04d}")
    df = None

    for ec in tqdm(data):
        if ec['lmfdb_iso'] in isog_labels:  # one per isogeny class
            continue
        isog_labels.add(ec['lmfdb_iso'])
        ec_sage = EllipticCurve(QQ, ec['ainvs'])
        aps = list(ec_sage.aplist(pmax))
        rank = ec['rank']
        if r is not None and rank not in r:
            continue
        row = [ec['lmfdb_iso'], rank] + aps
        if df is None:
            df = pl.DataFrame([row], schema=columns)
        else:
            df.extend(pl.DataFrame([row], schema=columns))

    if save_csv:
        df.write_csv(f"ec_data_N{max_N}_ap{num_ap}.csv")
    return df

def X_y(df, label, num_ap):
    """
    Given a polars dataframe df, return the feature matrix X and target vector y.
    The features are the first num_ap coefficients a_p of the L-series.
    The target is the column specified by label.
    """
    columns_ = [f"a_{p:04d}" for p in Primes()[:num_ap]]
    X = df.select(columns_)
    y = df.select(label)
    return X, y

Q1. On LMFDB, how many elliptic curves of rank 0 or 1 and conductor $\le$ 10000?

Q2. How about the number of isogeny classes?

In [4]:
data_r0 = query_data(1, 10000, r=0)
data_r1 = query_data(1, 10000, r=1)
data = data_r0 + data_r1
print(f"Number of elliptic curves of rank 0 or 1 and conductor <= 10000: {len(data)}")

Number of elliptic curves of rank 0 or 1 and conductor <= 10000: 62298


In [5]:
isog_labels = set()
for ec in tqdm(data):
    if ec['lmfdb_iso'] in isog_labels:
        continue
    isog_labels.add(ec['lmfdb_iso'])
print(f"Number of isogeny classes of rank 0 or 1 and conductor <= 10000: {len(isog_labels)}")

100%|██████████| 62298/62298 [00:00<00:00, 1168287.07it/s]

Number of isogeny classes of rank 0 or 1 and conductor <= 10000: 36072


Q3. Let $E$ be the following elliptic curve
$$
y^2+xy+y=x^3-208x-1122
$$

1. What is the conductor, analytic / algebraic rank, torsion subgroup of the curve?
2. What is $a_{1987}(E)$?
3. Find the LMFDB label of the curve. (You can search on [this webpage](https://www.lmfdb.org/EllipticCurve/Q/).)

Hint: To find useful Sage functions, you can read [reference manual](https://doc.sagemath.org/html/en/reference/index.html). LMFDB also shows Sage commands.

In [6]:
E = EllipticCurve([1, 0, 1, -208, -1122])

conductor = E.conductor()
analytic_rank = E.analytic_rank()
rank = E.rank()
torsion_subgroup = E.torsion_subgroup()
a_1987 = E.ap(1987)

print(f"Conductor: {conductor}")
print(f"Analytic Rank: {analytic_rank}")
print(f"Algebraic Rank: {rank}")
print(f"Torsion Subgroup: {torsion_subgroup}")
print(f"a_1987: {a_1987}")

# LMFDB label: 130.a1 (https://www.lmfdb.org/EllipticCurve/Q/130/a/1)

Conductor: 130
Analytic Rank: 1
Algebraic Rank: 1
Torsion Subgroup: Torsion Subgroup isomorphic to Z/2 associated to the Elliptic Curve defined by y^2 + x*y + y = x^3 - 208*x - 1122 over Rational Field
a_1987: 32


Q4. Train a logistic regression model that distinguish (isogeny classes of) elliptic curves of rank 0 or 1 and conductor $\le$ 10000.

- Radomly split the data into train (70\%) and test (30\%) set.
- Use The first 300 $a_{p}(E)$'s as features.
- The resulting model may achieve an accuracy > 99\%.

In [7]:
df = get_df(10000, 300, r=[0, 1], save_csv=False)
print(df.head())

100%|██████████| 64687/64687 [01:01<00:00, 1057.40it/s]


shape: (5, 302)
┌────────────┬──────┬────────┬────────┬───┬────────┬────────┬────────┬────────┐
│ isog_label ┆ rank ┆ a_0002 ┆ a_0003 ┆ … ┆ a_1951 ┆ a_1973 ┆ a_1979 ┆ a_1987 │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆   ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ str        ┆ i64  ┆ i64    ┆ i64    ┆   ┆ i64    ┆ i64    ┆ i64    ┆ i64    │
╞════════════╪══════╪════════╪════════╪═══╪════════╪════════╪════════╪════════╡
│ 11.a       ┆ 0    ┆ -2     ┆ -1     ┆ … ┆ -23    ┆ 79     ┆ 30     ┆ -22    │
│ 14.a       ┆ 0    ┆ -1     ┆ -2     ┆ … ┆ -64    ┆ -60    ┆ 30     ┆ -82    │
│ 15.a       ┆ 0    ┆ -1     ┆ -1     ┆ … ┆ 32     ┆ -42    ┆ 60     ┆ 44     │
│ 17.a       ┆ 0    ┆ -1     ┆ 0      ┆ … ┆ -48    ┆ 6      ┆ 64     ┆ 44     │
│ 19.a       ┆ 0    ┆ 0      ┆ -2     ┆ … ┆ 8      ┆ -9     ┆ -48    ┆ -19    │
└────────────┴──────┴────────┴────────┴───┴────────┴────────┴────────┴────────┘


In [8]:
X, y = X_y(df, "rank", 300)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=float(0.3), random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
clr = classification_report(y_test, y_pred, digits=4)
print(clr)

              precision    recall  f1-score   support

           0     0.9943    0.9874    0.9908      4921
           1     0.9896    0.9953    0.9924      5901

    accuracy                         0.9917     10822
   macro avg     0.9919    0.9913    0.9916     10822
weighted avg     0.9917    0.9917    0.9917     10822



Q5. Do your own experiments (different model, different data, different problem, etc.).
You can modify `get_df` to generate new data with other information.
Check out the `Underlying data` section of LMFDB.